# ParaView Test
This notebook is useful as an extremely minimal, "hello world"-type example. It creates a synthetic data source and computes an isosurface mesh from it. Then, it creates a ParaView display widget showing the primary render view. The notebook further demonstrates how the same view can be displayed multiple times, and how multiple RenderViews can be handled.

In [ ]:
from paraview.simple import *
sphere = Sphere(ThetaResolution=16, PhiResolution=32)
shrink = Shrink(sphere)
Show(shrink)

from ipyparaview.widgets import PVDisplay
disp = PVDisplay(GetActiveView())
display(disp)

Since we have access to the ParaView state from inside the kernel, we can easily inspect and set properties of both the pipeline and display objects. Let's create a simple callback function to change the ShrinkFactor the shrink filter, and create a Jupyter widget to control it interactively.

Note that we also display our widget again in case we've gotten too far from the original display to see it properly. Because of how Jupyter widgets are structured, we can display the same widget multiple times without creating multiple copies of it. This makes structuring your notebooks easier by letting you display the widget wherever it is relevant.

In [ ]:
# Interact from ipywidgets offers us a simple way to interactively control values with a callback function
from ipywidgets import interact

# sets the isosurface value, triggers a pipeline update, and renders the result
def setShrink(factor):
    shrink.ShrinkFactor = factor
    
interact(setShrink, factor=shrink.ShrinkFactor, min=0.0, max=1.0)
display(disp)

If you want to visualize some different data, or visualize your data in a different way, you can create another RenderView and PVDisplay for it.

In [ ]:
renv2 = CreateView('RenderView')

wavelet = Wavelet()

contour = Contour(Input=wavelet)
contour.ContourBy = ['POINTS', 'RTData']
contour.Isosurfaces = [157]

Show(contour, renv2)

PVDisplay(renv2)

It is little hard to guess the shape of the wavelet data, isn't it? So we'll add a toggle for RTX to improve the perception of shape via physically-based rendering with global illumination.

In [ ]:
def toggleRTX(rtxon):
    renv2.EnableRayTracing = 1 if rtxon else 0
interact(toggleRTX, rtxon=False)

renv2.BackEnd = 'OptiX pathtracer'
renv2.Denoise = 1 #turn on denoising
renv2.SamplesPerPixel = 4 #need at least 4 accumulated frames to trigger denoising